In [1]:
from sumolib import checkBinary  # noqa
import sumolib
import traci  # noqa
import os
import sys
from pathlib import Path
import traci

try:
    dir = Path("Simulation_Environment\Main MADDPG")
    sumoBinary = checkBinary('sumo')
    # sumoBinary = checkBinary('sumo-gui')
    # we need to import python modules from the $SUMO_HOME/tools directory
    if 'SUMO_HOME' in os.environ:
        tools = os.path.join(os.environ['SUMO_HOME'], 'tools')
        sys.path.append(tools)
    else:
        sys.exit("please declare environment variable 'SUMO_HOME'")
    traci.start([sumoBinary, "-c", f"{dir}\osm.sumocfg",
                            "--tripinfo-output", f"{dir}\\Results\\tripinfo.xml",  "--start"])
    #start net
    net = sumolib.net.readNet(Path(f"{dir}\\osm.net.xml"))
except Exception as e:
    print(e)

print("Traci and sumolib Net is running in background...")


Traci and sumolib Net is running in background...


In [3]:
trafficlight = traci.trafficlight.getIDList()[0]

phase_idx = int(0)
duration=float(60)

# #Set the time on phase
# traci.trafficlight.setPhase(trafficlight, phase_idx)
# traci.trafficlight.setPhaseDuration(trafficlight, int(duration))

traci.trafficlight.getCompleteRedYellowGreenDefinition(trafficlight)[0].getPhases()[0].duration = duration

traci.trafficlight.getAllProgramLogics(trafficlight)

(Logic(programID='0', type=0, currentPhaseIndex=0, phases=(Phase(duration=33.0, state='rrrrGGGrrrrrGGGg', minDur=33.0, maxDur=33.0, next=()), Phase(duration=6.0, state='rrrryyyrrrrryyyg', minDur=6.0, maxDur=6.0, next=()), Phase(duration=33.0, state='rrrrrrrGrrrrrrrG', minDur=33.0, maxDur=33.0, next=()), Phase(duration=6.0, state='rrrrrrryrrrrrrry', minDur=6.0, maxDur=6.0, next=()), Phase(duration=33.0, state='GGGgrrrrGGGgrrrr', minDur=33.0, maxDur=33.0, next=()), Phase(duration=6.0, state='yyyyrrrryyyyrrrr', minDur=6.0, maxDur=6.0, next=())), subParameter={}),)

In [9]:
program = traci.trafficlight.getAllProgramLogics(trafficlight)[0]
program.phases[0].duration = 60
program
traci.trafficlight.setProgramLogic(trafficlight)

Logic(programID='0', type=0, currentPhaseIndex=0, phases=(Phase(duration=60, state='rrrrGGGrrrrrGGGg', minDur=33.0, maxDur=33.0, next=()), Phase(duration=6.0, state='rrrryyyrrrrryyyg', minDur=6.0, maxDur=6.0, next=()), Phase(duration=33.0, state='rrrrrrrGrrrrrrrG', minDur=33.0, maxDur=33.0, next=()), Phase(duration=6.0, state='rrrrrrryrrrrrrry', minDur=6.0, maxDur=6.0, next=()), Phase(duration=33.0, state='GGGgrrrrGGGgrrrr', minDur=33.0, maxDur=33.0, next=()), Phase(duration=6.0, state='yyyyrrrryyyyrrrr', minDur=6.0, maxDur=6.0, next=())), subParameter={})

In [8]:
import traci

traci.close()

In [ ]:

phases = traci.trafficlight.getCompleteRedYellowGreenDefinition('recto_osmena_tl')[0].getPhases()
min_val = min([i.duration for i in phases])
duration = [i.duration for i in phases if 'y' not in i.state]
duration

In [ ]:
import numpy as np
x = np.round(np.hstack([[1,2,3],[4,5,6]])/5)
x

In [ ]:
tls =  'recto_osmena_tl'
traci.trafficlight.getRedYellowGreenState(tls)
traci.trafficlight.getCompleteRedYellowGreenDefinition(tls)[0].getPhases()[0].duration = 34
traci.trafficlight.getCompleteRedYellowGreenDefinition(tls)[0].getPhase()